# ReAct - Reasoning an Acting

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [17]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from sympy import python

llm = ChatOpenAI(model_name='gpt-4o', temperature=0)

template = '''
Answer the following question as best tyou can.
Question: {question}
'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = hub.pull('hwchase17/react')

python_repl = PythonREPLTool()

python_repl_tool = Tool(
    name = 'Python REPL',
    func=python_repl._run,
    description='Usefull when you nedd use Python to answer a question, You should input Python code.'
)

api_wrapper = WikipediaAPIWrapper()

wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

search = DuckDuckGoSearchRun()

duck_duck_go_tool = Tool(
    name='DuckDuckGo search',
    func=search.run,
    description='Useful when yopu ned to perform a internet search to find information that another tool can\'t provide.'
)

tools = [python_repl_tool, wikipedia_tool, duck_duck_go_tool]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [18]:
question = 'Generate teh firt 20 numbers in the Fibonacci series.'

output = agent_executor.invoke({
    'input': prompt_template.format(
        question= question
    )
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


To generate the first 20 numbers in the Fibonacci series, I will use the Python REPL to write a script that calculates and prints these numbers.

Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    fib_series = [0, 1]
    while len(fib_series) < n:
        fib_series.append(fib_series[-1] + fib_series[-2])
    return fib_series

# Generate the first 20 numbers in the Fibonacci series
fibonacci(20)
```
Observation: The Python code successfully generated the first 20 numbers in the Fibonacci series.

Final Answer: The first 20 numbers in the Fibonacci series are:
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181

> Finished chain.


In [20]:
print(output['output'])

The first 20 numbers in the Fibonacci series are:
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181
